In [ ]:
%env CUDA_VISIBLE_DEVICES=''

In [ ]:
from boardlaw.mcts.tests import *; test_root_one_node('cpu')

In [ ]:
from boardlaw.arena.refine import *; evaluate('2021-02-06 12-16-42 wan-ticks', -1)

In [ ]:
import torch.utils.cpp_extension

torch.utils.cpp_extension.load?

In [ ]:
d.type

In [ ]:
evaluate('2021-02-06 12-16-42 wan-ticks', -1)

In [ ]:
from pavlov import runs
runs.pandas().query('description == "main/3"')

In [ ]:
from pavlov import stats
stats.pandas('2021-02-06 12-16-42 wan-ticks', 'elo-mohex', 'μ')